In [36]:
import datetime
import pandas as pd
from influxdb import DataFrameClient
import matplotlib
import matplotlib.pyplot as plt
import json
import pytz
import numpy as np
import yaml
import time
%matplotlib inline

In [2]:
with open('config.yaml') as fp:
    config = yaml.safe_load(fp)
database_config = config.get('database')

In [3]:
with open('uuid_map.json') as fp:
    uuid_map = json.load(fp)

In [4]:
client = DataFrameClient(
    host=database_config.get('host'), 
    port=database_config.get('port'), 
    username=database_config.get('username'), 
    password=database_config.get('password'), 
    database=database_config.get('database'), 
    ssl=database_config.get('ssl'), 
    verify_ssl=database_config.get('verify_ssl')
)

In [5]:
tz_local = pytz.timezone('America/Los_Angeles')
tz_utc = pytz.timezone('UTC')

In [182]:
device_dict = {
    'thermostat_east': 'xbos/flexstat/thermostat_east',
    'thermostat_west': 'xbos/flexstat/thermostat_west',
    'building_meter': 'xbos/wattnode/building_main',
    'freezer_compressor_fan_meter': 'xbos/wattnode/fre_comp_evapfan',
    'ref_compressor_meter': 'xbos/wattnode/ref_comp',
    'ref_fan_meter': 'xbos/wattnode/ref_evapfan',
    'hvac_west_meter': 'xbos/wattnode/hvac_west_comp',
    'hvac_east_meter': 'xbos/wattnode/hvac_east_comp',
    'rtac': 'xbos/rtac/blr',
    'weather': 'xbos/dark_sky/blr',
    'freezer': 'xbos/parker/freezer',
    'refrigerator': 'xbos/parker/refrigerator',
    'price_forecast': 'xbos/dr_signals',
#     'weather_forecast': 'xbos/dark_sky/blr/prediction'
}

In [147]:
def parse_dt_utc(dt):
    return tz_local.localize(datetime.datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")).astimezone(tz_utc)

In [173]:
def get_collection_data(client, collection, st, et, forecast=False):
    st_utc = st.strftime("%Y-%m-%dT%H:%M:%SZ")
    et_utc = et.strftime("%Y-%m-%dT%H:%M:%SZ")
    query = "select * from timeseries where \"collection\" = '%s' and time>= '%s' and time <= '%s'" % (collection, st_utc, et_utc)
    df = client.query(query)['timeseries']
    
    df_list = []
    prediction_time_df = None
    for var_name in df.name.unique():
        if not forecast:
            df2 = df.loc[df.name == var_name][['value']]
            df2.columns = [var_name]
            df_list.append(df2)
        else:
            df2 = df.loc[df.name == var_name][['prediction_time', 'value']].reset_index().sort_values(by=['index', 'prediction_time'])
            df2 = df2.set_index(['index'])
            df2.columns = ['prediction_time', var_name]
            if prediction_time_df is None:
                prediction_time_df = df2.prediction_time
            df_list.append(df2[[var_name]])
    if forecast:
        df_list = [prediction_time_df] + df_list
    final_df = pd.concat(df_list, axis=1)
    return final_df

In [174]:
st = "2020-08-11 00:00:00"
et = "2020-08-11 23:59:59"

In [175]:
st_utc = parse_dt_utc(st)
et_utc = parse_dt_utc(et)
df_dict = {}

In [181]:
for device in device_dict:
    print(device)
    if not device in df_dict:
        collection = device_dict[device]
        
        st_part = st_utc
        et_part = st_part + datetime.timedelta(seconds=3599)
        completed = False
        
        df_part_list = []

        while not completed:
            print(st_part)
            if et_part >= et_utc:
                completed = True
                et_part = et_utc

            forecast = False
            delay = 5
            if device.endswith('_forecast'):
                forecast = True
                delay = 10
                
                
            df = get_collection_data(client, collection, st_part, et_part, forecast=forecast)
            df_part_list.append(df)
            time.sleep(delay)
            
            st_part = et_part + datetime.timedelta(seconds=1)
            et_part = st_part + datetime.timedelta(seconds=3599)
        df_dict[device] = pd.concat(df_part_list, axis=0, sort=True)
        df_dict[device].to_csv('sample_data/{0}.csv'.format(device))

thermostat_east
thermostat_west
building_meter
freezer_compressor_fan_meter
ref_compressor_meter
ref_fan_meter
hvac_west_meter
hvac_east_meter
rtac
weather
freezer
refrigerator
price_forecast
2020-08-11 07:00:00+00:00
2020-08-11 08:00:00+00:00
2020-08-11 09:00:00+00:00
2020-08-11 10:00:00+00:00
2020-08-11 11:00:00+00:00
2020-08-11 12:00:00+00:00
2020-08-11 13:00:00+00:00
2020-08-11 14:00:00+00:00
2020-08-11 15:00:00+00:00
2020-08-11 16:00:00+00:00
2020-08-11 17:00:00+00:00
2020-08-11 18:00:00+00:00
2020-08-11 19:00:00+00:00
2020-08-11 20:00:00+00:00
2020-08-11 21:00:00+00:00
2020-08-11 22:00:00+00:00
2020-08-11 23:00:00+00:00
2020-08-12 00:00:00+00:00
2020-08-12 01:00:00+00:00
2020-08-12 02:00:00+00:00
2020-08-12 03:00:00+00:00
2020-08-12 04:00:00+00:00
2020-08-12 05:00:00+00:00
2020-08-12 06:00:00+00:00
weather_forecast
2020-08-11 07:00:00+00:00
2020-08-11 08:00:00+00:00
2020-08-11 09:00:00+00:00
2020-08-11 10:00:00+00:00
2020-08-11 11:00:00+00:00
2020-08-11 12:00:00+00:00
2020-08-11 

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))